In [1]:
#https://github.com/wolverton-research-group/qmpy/blob/master/qmpy/data/thermodata/ssub.dat
import pandas as pd
df=pd.read_csv('ssub.dat',sep=' ',header=None)
df.columns=['formula','formula_energy']

In [2]:
df

,formula,formula_energy
0,AlKO6Si2,-3.144727e+00
1,Al2H4O9Si2,-2.500142e+00
2,Al2H4O9Si2,-2.499380e+00
3,AlHO2,-2.589914e+00
4,AlHO2,-2.566077e+00
...,...,...
2085,AsS,-1.763221e-01
2086,As7Re3,-9.887016e-02
2087,As8Ni11,-4.222273e-01
2088,Au,1.605726e-09


In [3]:
#https://stackoverflow.com/questions/12497402/remove-duplicates-by-columns-a-keeping-the-row-with-the-highest-value-in-column
df=df.sort_values('formula_energy', ascending=True).drop_duplicates('formula').sort_index()
df=df.sample(frac=1)
df['id']=df.index

In [4]:
df

,formula,formula_energy,id
1982,AsK3O4,-2.161922,1982
517,CdN2O6,-0.526223,517
1800,Re5Si3,-0.203811,1800
787,CrSi,-0.367932,787
2025,B2V3,-0.629646,2025
...,...,...,...
1349,Li2O3Si,-2.849311,1349
1621,O2Se,-0.779048,1621
1705,O5Rb2Si2,-2.849610,1705
1684,O4Pb2Si,-2.040088,1684


In [5]:
df.to_csv('ssub.csv',index=False)

In [6]:
from jarvis.db.jsonutils import dumpjson
#for jarvis-tools
mem=[]
for i,ii in df.iterrows():
    info={}
    info['id']=ii['id']
    info['formula_energy']=ii['formula_energy']
    info['formula']=ii['formula']
    mem.append(info)
dumpjson(filename='ssub.json',data=mem)

In [7]:
test_size = 0.2
val_size = 0.2
n_train = int(len(df)*(1-(val_size+test_size)))
n_val = int(len(df)*val_size)
n_test = int(len(df)*test_size)
#for leaderboard
mem={}
train={}
val={}
test={}
train_df = df[:n_train]
val_df = df[n_train:n_train+n_val]
test_df = df[n_train+n_val:n_train+n_val+n_test]
for i,ii in train_df.iterrows():
    train[str(ii['id'])]=ii['formula_energy']
for i,ii in val_df.iterrows():
    val[str(ii['id'])]=ii['formula_energy']
for i,ii in test_df.iterrows():
    test[str(ii['id'])]=ii['formula_energy']
mem['train']=train
mem['val']=val
mem['test']=test
fname='ssub_formula_energy.json'
dumpjson(data=mem,filename=fname)

In [8]:
len(train_df), len(val_df), len(test_df)

(1036, 345, 345)

In [9]:
import data_utils
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

elements = ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K',
 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se',
 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In',
 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd',
 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au',
 'Hg', 'Tl', 'Pb', 'Bi', 'Ac','Th', 'Pa', 'U', 'Np', 'Pu']

def formula_to_ef(df):
    df['comp_dict'] = df['formula'].apply(lambda x: data_utils.parse_formula(x))
    df['comp_fractions'] = df['comp_dict'].apply(lambda x: data_utils.get_fractions(x))
    df = df[~df['comp_fractions'].isnull()]

    for i,e in enumerate(elements):
        df[e] = [ x[i] for x in df['comp_fractions']]

    X_df = np.array(df[elements],dtype='float')    
    y_df = np.array(df['formula_energy'],dtype='float')
    list_df = list(df['id'])

    return X_df, y_df, list_df

In [10]:
X_train, y_train, train_ids = formula_to_ef(train_df)
X_val, y_val, val_ids = formula_to_ef(val_df)
X_test, y_test, test_ids = formula_to_ef(test_df)

/tmp/ipykernel_61438/1972034185.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comp_dict'] = df['formula'].apply(lambda x: data_utils.parse_formula(x))
/tmp/ipykernel_61438/1972034185.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comp_fractions'] = df['comp_dict'].apply(lambda x: data_utils.get_fractions(x))
/tmp/ipykernel_61438/1972034185.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [11]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

2023-05-14 17:37:23.226028: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [12]:
#Network from "Cross-property deep transfer learning framework for enhanced predictive analytics on small materials data"
#Link to paper: https://www.nature.com/articles/s41467-021-26921-5
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = Activation('relu')(layer_1)

layer_2 = Dense(1024)(layer_1)
layer_2 = Activation('relu')(layer_2)

layer_3 = Dense(1024)(layer_2)
layer_3 = Activation('relu')(layer_3)

layer_4 = Dense(1024)(layer_3)
layer_4 = Activation('relu')(layer_4)
layer_4 = Dropout(0.2)(layer_4, training=False)

layer_5 = Dense(512)(layer_4)
layer_5 = Activation('relu')(layer_5)

layer_6 = Dense(512)(layer_5)
layer_6 = Activation('relu')(layer_6)

layer_7 = Dense(512)(layer_6)
layer_7 = Activation('relu')(layer_7)
layer_7 = Dropout(0.1)(layer_7, training=False)

layer_8 = Dense(256)(layer_7)
layer_8 = Activation('relu')(layer_8)

layer_9 = Dense(256)(layer_8)
layer_9 = Activation('relu')(layer_9)

layer_10 = Dense(256)(layer_9)
layer_10 = Activation('relu')(layer_10)
layer_10 = Dropout(0.3)(layer_10, training=False)

layer_11 = Dense(128)(layer_10)
layer_11 = Activation('relu')(layer_11)

layer_12 = Dense(128)(layer_11)
layer_12 = Activation('relu')(layer_12)

layer_13 = Dense(128)(layer_12)
layer_13 = Activation('relu')(layer_13)
layer_13 = Dropout(0.2)(layer_13, training=False)

layer_14 = Dense(64)(layer_13)
layer_14 = Activation('relu')(layer_14)

layer_15 = Dense(64)(layer_14)
layer_15 = Activation('relu')(layer_15)

layer_16 = Dense(32)(layer_15)
layer_16 = Activation('relu')(layer_16)

out_layer = Dense(1)(layer_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


2023-05-14 17:37:26.828441: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-05-14 17:37:28.441836: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.77GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-14 17:37:28.442079: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.77GiB deviceMemoryBandwidth: 836.37GiB/s
2023-05-14 17:37:28.442117: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-05-14 17:37:28.479196: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.

In [13]:
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

2023-05-14 17:37:30.655783: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-05-14 17:37:30.656461: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2600000000 Hz


Epoch 1/3000


2023-05-14 17:37:31.237601: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-05-14 17:37:31.768937: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


33/33 - 2s - loss: 1.3433 - mean_absolute_error: 1.3433 - val_loss: 0.9858 - val_mean_absolute_error: 0.9858
Epoch 2/3000
33/33 - 0s - loss: 0.8262 - mean_absolute_error: 0.8262 - val_loss: 0.6358 - val_mean_absolute_error: 0.6358
Epoch 3/3000
33/33 - 0s - loss: 0.6136 - mean_absolute_error: 0.6136 - val_loss: 0.4949 - val_mean_absolute_error: 0.4949
Epoch 4/3000
33/33 - 0s - loss: 0.4972 - mean_absolute_error: 0.4972 - val_loss: 0.4817 - val_mean_absolute_error: 0.4817
Epoch 5/3000
33/33 - 0s - loss: 0.4451 - mean_absolute_error: 0.4451 - val_loss: 0.4228 - val_mean_absolute_error: 0.4228
Epoch 6/3000
33/33 - 0s - loss: 0.4126 - mean_absolute_error: 0.4126 - val_loss: 0.3645 - val_mean_absolute_error: 0.3645
Epoch 7/3000
33/33 - 0s - loss: 0.3318 - mean_absolute_error: 0.3318 - val_loss: 0.3366 - val_mean_absolute_error: 0.3366
Epoch 8/3000
33/33 - 0s - loss: 0.3047 - mean_absolute_error: 0.3047 - val_loss: 0.3484 - val_mean_absolute_error: 0.3484
Epoch 9/3000
33/33 - 0s - loss: 0.275

In [14]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

0.19651267243325324


In [15]:
f=open('SinglePropertyPrediction-test-formula_energy-ssub-AI-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k)+'\n'
    f.write(line)
f.close()